In [ ]:
# Import the Earth Engine API
import ee

# Import the geemap library for interactive mapping
import geemap

import warnings

# Filter out FutureWarning messages to keep the output clean
warnings.simplefilter(
    action='ignore', 
    category=FutureWarning
)

In [2]:
# Authenticate and initialize the Earth Engine session
try:
    # Try to initialize the existing session
    ee.Initialize()
except Exception as e:
    # If initialization fails, trigger the authentication flow
    ee.Authenticate()
    # Initialize after authentication
    ee.Initialize()

In [3]:
# Create an interactive map instance using geemap
# We leave the center and zoom default for now; we will center on the data later
m = geemap.Map()

In [9]:
# Define the GLANCE dataset asset ID
# This points to the version 1 collection hosted on GEE
dataset_id = "projects/GLANCE/DATASETS/V001"

# Load the image collection from the Earth Engine catalog
dataset = ee.ImageCollection(
    dataset_id
)

In [7]:
# Process the dataset to get a single image for visualization
# 1. Select the Land Cover ('LC') band
# 2. Filter for the year 2019
# 3. Create a mosaic to flatten the collection into one image
lulc_2019 = dataset.select(
    'LC'
).filterDate(
    '2019-01-01',
    '2019-12-31'
).mosaic()

In [5]:
# Define visualization parameters for the 7 Land Cover classes
# Based on the official GLANCE documentation
vis_params = {
    'min': 1,
    'max': 7,
    'palette': [
        '0000ff',  # 1: Water (Blue)
        'ffffff',  # 2: Ice/Snow (White)
        'ff0000',  # 3: Developed (Red)
        'a52a2a',  # 4: Barren/Sparsely Vegetated (Brown)
        '006400',  # 5: Trees (Dark Green)
        'bdb76b',  # 6: Shrub (Olive)
        '90ee90',  # 7: Herbaceous (Light Green)
    ]
}

In [10]:
# Add the processed GLANCE layer to the interactive map
m.add_layer(
    lulc_2019,
    vis_params,
    'GLANCE LULC 2019'
)

# Manually set the center of the map instead of calculating it from the mosaic
# We use coordinates for South America as an example, but you can change them
# Arguments: longitude, latitude, zoom level
m.set_center(
    -55.0,
    -10.0,
    4
)

# Display the map object
m

Map(center=[-10.0, -55.0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…